In [145]:
import os
import json
from litellm import completion
from pydantic import BaseModel, ValidationError

In [173]:
##############################################################################
# 1) DEFINE AGENTS (PI, Critic, Scientist, etc.) WITH SEPARATE MODELS & PROMPTS
##############################################################################

# Each agent has:
# - name
# - model (swappable)
# - system_prompt (the agent's role/expertise/goal, etc.)

AGENTS = {
    "principal_investigator": {
        "name": "Principal Investigator",
        "model": "openai/gpt-4o",
        "system_prompt": """You are a Principal Investigator. 
        Your expertise is in applying artificial intelligence to biomedical research. 
        Your goal is to perform research in your area of expertise that maximizes the scientific impact of the work. 
        Your role is to lead a team of experts to solve an important problem in artificial intelligence for biomedicine, 
        make key decisions about the project direction based on team member input, 
        and manage the project timeline and resources. Be focused and provide concise answers. Reply in a conversational tone and in paragraph form.""",
    },
    "scientific_critic": {
        "name": "Scientific Critic",
        # "model": "anthropic/claude-3-5-sonnet-20240620",
        "model": "openai/gpt-4o",
        "system_prompt": """You are a Scientific Critic. 
        Your expertise is in providing critical feedback for scientific research. 
        Your goal is to ensure that proposed research projects and implementations are rigorous, 
        detailed, feasible, and scientifically sound. 
        Your role is to provide critical feedback to identify and correct all errors and demand 
        that scientific answers that are maximally complete and detailed but 
        simple and not overly complex. Be focused and provide concise answers. Reply in a conversational tone and in paragraph form.""",
    },
    "biologist": {
        "name": "Biologist",
        #"model": "mistral/mistral-tiny",
        "model": "openai/gpt-4o",
        "system_prompt": """You are a Biologist with deep knowledge of molecular biology, 
        genetic engineering, etc. Offer domain-specific insights. Be focused and provide concise answers. Reply in a conversational tone and in paragraph form."""
    },
    "computer_scientist": {
        "name": "Computer Scientist",
        "model": "openai/gpt-4o",
        "system_prompt": """You are a Computer Scientist with deep knowledge of computer science, 
        artificial intelligence, etc. Offer domain-specific insights. Be focused and provide concise answers. Reply in a conversational tone and in paragraph form."""
    },
    "computational_biologist": {
        "name": "Computational Biologist",
        "model": "openai/gpt-4o",
        "system_prompt": """You are a Computational Biologist with deep knowledge of computational biology, 
        molecular biology, etc. Offer domain-specific insights. Be focused and provide concise answers. Reply in a conversational tone and in paragraph form."""
    },
# possibly a final summary agent
    "summary_agent": {
        "model": "openai/gpt-4o",
        "system_prompt": """You are the Summary Agent; 
        your job is to read the entire conversation and produce a final summary.
        Also provide an answer to the user's question.

        Format as follows:
        Summary: <summary>
        Answer: <answer>
        """
    },
    # Add more agents as needed...
}

In [174]:
##############################################################################
# 2) HELPER FUNCTION: CALL AN AGENT
##############################################################################

def call_agent(agent_key: str, conversation_history: str) -> str:
    """
    Calls the specified agent (by key in AGENTS) with the conversation so far.
    The agent's system prompt is used, plus the `conversation_history` is
    appended as the user input. Returns the agent's text response.
    """
    agent_config = AGENTS[agent_key]
    agent_system_prompt = agent_config["system_prompt"]
    agent_model = agent_config["model"]

    # We pass a minimal conversation:
    # - system: agent’s role
    # - user: conversation_history so far (or direct question, instructions)
    messages = [
        {"role": "system", "content": agent_system_prompt},
        {"role": "user", "content": conversation_history},
    ]

    # Run litellm completion for the agent
    response = completion(
        model=agent_model,
        messages=messages,
        temperature=1,
        max_tokens=500,  # Increase as needed
    )

    # Return the agent's text
    return response.choices[0].message.content.strip()

In [175]:
# test the call_agent function
print(call_agent("principal_investigator", "What is the main goal of the project?"))


The main goal of our project is to develop a robust artificial intelligence model that can significantly enhance precision medicine through improved diagnostics and personalized treatment strategies. By leveraging vast biomedical datasets, we aim to create an AI system that not only predicts disease risks but also suggests tailored therapeutic approaches based on individual patient profiles, thereby maximizing treatment efficacy and minimizing adverse effects. Our work is poised to bridge the gap between AI innovation and practical healthcare applications, ultimately elevating patient outcomes and transforming the healthcare landscape.


In [176]:
###############################################################################
# 2) The Pydantic model describing the Orchestrator's response
###############################################################################

class OrchestratorResponse(BaseModel):
    agent: str
    rationale: str

In [177]:
def call_orchestrator(conversation_text: str) -> str:
    """
    The Orchestrator is just a normal GPT call (no function-calling).
    We dynamically create a prompt listing all the agent keys from AGENTS.
    Returns a JSON string we parse to figure out the next agent.
    """
    # Gather all agent keys into a comma-separated list
    agent_names_str = ", ".join(AGENTS.keys())

    # remove Principal Investigator and Summary Agent from the list of agents
    agent_names_str = ", ".join([name for name in agent_names_str.split(", ") if name not in ["principal_investigator", "summary_agent"]])

    # Dynamically build the system prompt with f-string
    orchestrator_prompt = f"""You are the Orchestrator. 
Read the conversation so far, then decide which agent should speak next. 
Your possible agents are: {agent_names_str}.

Output your choice in a JSON format, for example:
{{
  "agent": "scientific_critic",
  "rationale": "I want critical feedback next."
}}
Only output valid JSON and nothing else.
"""

    messages = [
        {"role": "system", "content": orchestrator_prompt},
        {"role": "user", "content": conversation_text},
    ]
    resp = completion(
        model="openai/gpt-4o",
        messages=messages,
        temperature=1,
        max_tokens=300,
    )
    return resp.choices[0].message.content.strip()

In [181]:
def run_conversation(agenda, conversation_length=3, max_rounds=3):
    """
    - We have 'max_rounds' total rounds of discussion.
    - Each round allows up to 'conversation_length' agent calls (not counting the PI's round-end synthesis).
    - At the end of each round, the PI agent synthesizes the points raised.
    - After the final round, the PI also provides a final summary.
    """
    conversation_history = f"The user wants to discuss: {agenda}\n\n"

    # principal investigator's initial response
    pi_reply = call_agent("principal_investigator", conversation_history)
    conversation_history += f"\n\n[principal_investigator]: {pi_reply}"
    print(f"[principal_investigator] => {pi_reply}")

    # 1) Round-based discussion
    for round_index in range(1, max_rounds + 1):
        # add print to the conversation history
        conversation_history += f"\n\n=== ROUND {round_index} of {max_rounds} ==="
        print(f"\n=== ROUND {round_index} of {max_rounds} ===")

        # Inner loop: up to 'conversation_length' calls (the orchestrator chooses who speaks)
        calls_this_round = 0
        while calls_this_round < conversation_length:
            # Ask the orchestrator who should speak next
            orch_json_str = call_orchestrator(conversation_history)
            try:
                orch_data = OrchestratorResponse.model_validate_json(orch_json_str)
                chosen_agent = orch_data.agent
                rationale = orch_data.rationale
            except (json.JSONDecodeError, ValidationError):
                # If invalid JSON from orchestrator, default to summary => break out
                chosen_agent = "summary_agent"
                rationale = "No valid JSON => final summary."

            # Otherwise, call the chosen agent
            agent_reply = call_agent(chosen_agent, conversation_history)
            conversation_history += f"\n[{chosen_agent}]: {agent_reply}"
            print(f"[{chosen_agent}] => {agent_reply}")
            calls_this_round += 1

        # 2) End of the round: The PI agent synthesizes what's been said.
        # This is the "PI round-end" step: "makes decisions based on agent input, and asks follow-up questions"
        pi_synthesis = call_agent("principal_investigator", 
            conversation_history + "\nNow please synthesize this round's points concisely, and ask a couple focused follow-up questions for the next round .")
        conversation_history += f"\n\n[principal_investigator (round synthesis)]: {pi_synthesis}"
        print(f"[principal_investigator (round synthesis)] => {pi_synthesis}")

        # If this was the last round, the PI also provides the final summary & agenda answers
        if round_index == max_rounds:
            # 3) Final summary from PI
            final_summary = call_agent("summary_agent", conversation_history)
            conversation_history += f"\n\n[principal_investigator (final summary)]: {final_summary}"
            print("\n=== FINAL SUMMARY ===")
            print(final_summary)
            break

    return conversation_history


In [183]:
final = run_conversation("Designing a new protein folding experiment.")

[principal_investigator] => Designing a new protein folding experiment is an exciting venture, especially given the potential impact on drug discovery and understanding biological processes. Given our expertise in artificial intelligence and biomedical research, we can leverage computational models like AlphaFold or Rosetta to predict protein structures and design experiments that test these predictions in real-world scenarios. It would be essential to involve computational biologists to fine-tune these models and compare their predictions against experimental data.

First, I suggest we outline the specific goals of our protein folding experiment. Do we aim to predict novel protein structures, understand the folding pathways better, or perhaps identify folding-related diseases? Once we clearly define these objectives, we can tailor our computational models to suit our specific needs.

Next, we'll need to gather input from our team, especially our biochemists, to narrow down the protein

In [184]:
with open("conversation_history.txt", "w") as f:
    f.write(final)
